In [1]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import RMSprop
from keras import backend as K
from keras.models import load_model
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Reloading The VGG16 Model and Freezing the Layers**

In [3]:
# VGG was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the VGG model without the FC layers
VGG_16 = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in VGG_16.layers:
    layer.trainable = False

# Defining a function for additional layers adding at the top of pre-trained model

In [4]:
def TopModelVGG_16(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

# Combining the models and printing the summary

In [5]:
num_classes = 5  # number of classes or the people whose data is to be inserted

FC_Head = TopModelVGG_16(VGG_16, num_classes)

model = Model(inputs = VGG_16.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

# **Importing dataset and using Image generator for Augmentation**

In [6]:
#path to training data
train_data_dir = '/content/drive/My Drive/Internity_Project/Dataset_face/train_data'
#path to validation data
validation_data_dir = '/content/drive/My Drive/Internity_Project/Dataset_face/val_data'

# Let's use some data augmentation 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 200 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


#  Training the Model

In [7]:
#Path where you have to save the model
checkpoint = ModelCheckpoint("/content/drive/My Drive/Internity_Project/vgg_16_saved_model.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 200
nb_validation_samples = 25

# We train 50 EPOCHS 
epochs = 50
batch_size = 8

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/50
25/25 [==============================] - 22s 893ms/step - loss: 1.7544 - accuracy: 0.3139 - val_loss: 1.4009 - val_accuracy: 0.5600

Epoch 00001: val_loss improved from inf to 1.40089, saving model to /content/drive/My Drive/Internity_Project/vgg_16_saved_model.h5
Epoch 2/50
25/25 [==============================] - 16s 644ms/step - loss: 1.0842 - accuracy: 0.6085 - val_loss: 0.6114 - val_accuracy: 0.7200

Epoch 00002: val_loss improved from 1.40089 to 0.61142, saving model to /content/drive/My Drive/Internity_Project/vgg_16_saved_model.h5
Epoch 3/50
25/25 [==============================] - 16s 644ms/step - loss: 0.6685 - accuracy: 0.7415 - val_loss: 0.3877 - val_accuracy: 0.8800

Epoch 00003: val_loss improved from 0.61142 to 0.38770, saving model to /content/drive/My Drive/Internity_Project/vgg_16_saved_model.h5
Epoch 4/50
25/25 [==============================] - 17s 677ms/step - loss: 0.6039 - accuracy: 0.7830 - val_loss: 0.4747 - val_accuracy: 0.7600

Epoch 00004: val_los